In [7]:


epsilon = 0

#alpha = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
#note= ranks start at 2, not 1
playercards = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
alpha = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [1]:
import math

MAX = 4


def choose(n, r, k):
    return math.factorial(n)//math.factorial(r)//math.factorial(k)//math.factorial(n-r-k)

def probTwocards(p1, p2, count1, count2, total): #p1: probability of first card, p2: probability of second, count1: number you want of first, count2: number you want of second, total: total num of cards
    max = MAX*2
    if count1+count2>max or count1+count2>total:
        return 0
    #elif total > max:
        #return choose(max, count1, count2)* (p1)**count1*(p2)**count2*(1-(p1+p2))**(max-count1-count2)
    else:
        return choose(total, count1, count2)* (p1)**count1*(p2)**count2*(1-(p1+p2))**(total-count1-count2)

    
def probOnecard(p, count, total):  #p: probability, count: number you want, total: total number of cards
    max = MAX
    if count>max or count>total:
        return 0
    #elif num > max:
        #return choose(max, count, 0)* (p)**count*(1-(p))**(max-count)
    else:
        return choose(total, count, 0)* (p)**count*(1-(p))**(total-count)

def twotype(count1, count2, card1, card2, num): #count: number you want, card: card index, num: number of cards
    p1 = (alpha[card1] + 1)/(sum(alpha)+1)
    p2 = (alpha[card2] + 1)/(sum(alpha)+1) 
    prob = 0
    for i in range(max(count1-playercards[card1], 0), 5-playercards[card1]):
        for j in range(max(count2-playercards[card2], 0), 5-playercards[card2]):
            prob = prob + probTwocards(p1, p2, i, j, num)
    
    return prob

def onetype(count, card, num):
    p = (alpha[card] + 1)/(sum(alpha)+1)
    prob=0
    for i in range(max(count-playercards[card], 0), 5-playercards[card]):
        prob = prob+probOnecard(p, i, num)
    return min(prob, 1)

def straight(card, num):
    p = (alpha[card] + 1)/(sum(alpha)+1)
    prob = 1
    for i in range(card, card+4):
        prob = prob*probOnecard(p, 1, num)
    return prob

In [55]:
#remember- some straights are impossible
twotypeprob = [[[0 for x in range(13)] for y in range(13)] for z in range(3)]
onetypeprob = [[0 for x in range(13)] for y in range(4)] 
straightprob = [0 for x in range(13)]

prob = [straightprob, onetypeprob, twotypeprob]

def updateTwoType(num):
    
    largest = twotype(2, 2, 0, 1, num)
    ind = [0, 0, 0]
    for k in range(0, 3):
        for i in range(0, 13):
            for j in range(0, 13):
                l = twotype(k+2, max(k+1, 2), i, j, num)
                twotypeprob[k][i][j] = l
                if(l> largest):
                    largest = l
                    ind = [k, i, j]
                    
def updateOneType(num):
    largestone = onetype(1, 0, num)
    indone = [0, 0]
    for k in range(0, 4):
        for i in range(0, 13):
            l = onetype(k, i, num)
            onetypeprob[k][i] = l
            if(l> largestone):
                largestone = l
                indone = [k, i]
            
def updateStraight(num):
    largest = straight(0, num)
    ind = 0
    for i in range(0, 13):
        l = straight(i, num)
        straightprob[i] = l
        if(l > largest):
            largest = l
            ind = i

def updateProb(num):
    updateTwoType(num)
    updateOneType(num)
    updateStraight(num)
    prob = [straightprob, onetypeprob, twotypeprob]
    
def checkProb(call):
    k = findCall(call)
    probCall = 0
    if k == 0:
        probCall = prob[k][call[2]]
    elif k == 1:
        probCall = prob[k][call[0]-k][call[2]]
    else:
        probCall = prob[k][call[0]-k][call[2]][call[3]]
    return probCall

In [63]:

def makeCall(call):
    k = findCall(call)
    call = [0, 0, 0, 0, 0] #5th term is prob
   
    if k==0:
        #same level
        level = call[2]
        for i in range(level+1, 11):
            p = checkProb([0, 0, i, i+5])
            if p > call[4]:
                call = [0, 0, i, i+5, p]
        
        #higher 2
        for i in range(3, 5):
            for j in range(0, 13):
                for k in range(0, 13):
                    
                    p = checkProb([i, i-1, j, k])
                    if p>call[4]:
                        call = [i, i-1, j, k, p]
        
        #higher 1
        for i in range(4, 5):
            for j in range(0, 13):
                p = checkProb([i, 0, j, 0])
                if p>call[4]:
                    call = [i, 0, j, 0, p]
    elif k==1:
        
        level = call[2]
        calllevel = call[0]
        #same level
        for i in range(level+1, 13):
            p = checkProb([calllevel, 0, i, 0])
            if p > call[4]:
                call = [calllevel, 0, i, 0, p]
        #straight
        if(calllevel<4):
             for i in range(0, 10):
                p = checkProb([0, 0, i, i+5])
                if p > call[4]:
                    call = [0, 0, i, i+5, p]
        #higher 2
        
        
        #higher 1
        for i in range(calllevel+1, 5):
            for j in range(0, 13):
                p = checkProb([i, 0, j, 0])
                if p > call[4]:
                        call = [i, 0, j, 0, p]
    else:
        level = call[2]
        calllevel = call[0]
        #same level
        
        
        #straight
        if(calllevel< 3):
             for i in range(0, 10):
                p = checkProb([0, 0, i, i+5])
                if p > call[4]:
                    call = [0, 0, i, i+5, p]
        #higher 2
        for i in range(calllevel+1, 5):
            for j in range(0, 13):
                for k in range(0, 13):
                    p = checkProb([i, max(i-1, 2), j, k])
                    if p > call[4]:
                        call = [i, max(i-1, 2), j, k, p]
        #higher 1
        for i in range(calllevel+1, 5):
            for j in range(0, 13):
                p = checkProb([i, 0, j, 0])
                if p > call[4]:
                        call = [i, 0, j, 0, p]
    
    return call
        

In [62]:
def makeDecision(call):
    probOppCall = checkProb(call)
    owncall = makeCall(call)
    probOwnCall = owncall.pop()
    if(probOppCall > probOwnCall):
        return 0
    else:
        return 1

In [8]:
callFactor = 1;
def updateParameters(call):
    alpha[call[2]] = alpha[call[2]]+callFactor*call[0]
    alpha[call[3]] = alpha[call[3]]+callFactor*call[1]
    if(call[0] == 0 and call[1] == 0):
        for i in range(call[2], call[3]):
            alpha[i] = alpha[i]+callFactor
            
    

In [40]:
def findCall(call):
    return (call[0]>0) + (call[1]>0)+0


In [11]:
alpha

[1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1]

In [59]:
updateProb(8)

In [60]:
makeDecision([3,2,4,5])

0.08953264060840249

In [64]:
prob

[[0.02233235375823221,
  0.02233235375823221,
  0.02233235375823221,
  0.020705668218364656,
  0.020705668218364656,
  0.020705668218364656,
  0.020705668218364656,
  0.020705668218364656,
  0.02233235375823221,
  0.02233235375823221,
  0.02233235375823221,
  0.02233235375823221,
  0.02233235375823221],
 [[0.9994500772318827,
   0.9994500772318827,
   0.9994500772318827,
   0.9963935519977674,
   0.9963935519977674,
   0.6322241224694021,
   0.8811623706917292,
   0.9963935519977674,
   0.9994500772318827,
   0.9994500772318827,
   0.9994500772318827,
   0.9994500772318827,
   0.9994500772318827],
  [0.588714272020701,
   0.588714272020701,
   0.588714272020701,
   0.7435039030100066,
   0.7435039030100066,
   0.6322241224694021,
   0.8811623706917292,
   0.7435039030100066,
   0.588714272020701,
   0.588714272020701,
   0.588714272020701,
   0.588714272020701,
   0.588714272020701],
  [0.2021393965278242,
   0.2021393965278242,
   0.2021393965278242,
   0.36416942952836523,
   0.36416

In [15]:
twotype(3, 2, 1, 2, 6, alpha)

0.0038820301783264737

In [16]:
twotype(3, 2, 1, 2, 14, alpha)

0.02230568510897728

In [135]:
probOnecard(0.5, 1, 10)+probOnecard(0.5, 2, 10)+probOnecard(0.5, 3, 10)+probOnecard(0.5, 4, 10)

0.9375

In [16]:
onetype(2, 10, 10)

0.2622661335000001